In [ ]:
from google.colab import drive
drive.mount("/data")

Mounted at /data


In [ ]:
cd /data/MyDrive/Dataset

/data/MyDrive/Dataset


In [ ]:
import numpy as np
import pandas as pd
import json

In [ ]:
data_vk = pd.read_csv("vk_texts_full.csv")
data_vk.head()

,Unnamed: 0,text,vk_wall_id,vk_post_id,attachment_types,n_unique_attachments,has_link
0,139124,👌Новая поступления 🥳🥳🥳 \n Цена по ниже \n...,-192083416,9857,['photo'],0,False
1,139125,💞💞💞💞💞💞💞💞💞💞💞💞\n💥💥Новое поступление💥💥\n\n🌹КОМПЛЕ...,-192083416,9858,['photo'],0,False
2,139126,🌹ПОЛУЧИЛИ НОВЫЕ СОРОЧКИ\n🌹МАТЕРИАЛ 💯% ХЛОПОК\n...,-192083416,9860,['photo'],0,False
3,139127,КУРТКИ ДЛИННЫЕ ЗИМНЕЕ \nРазмер 42-44-46-48-50\...,-192083416,9862,['photo'],0,False
4,139128,ГРАНДИОЗНАЯ РАСПРОДАЖА \nТакова ещё не было на...,-192083416,9863,['photo'],0,False


In [ ]:
data_vk.drop(columns=["Unnamed: 0", "attachment_types", "n_unique_attachments", "has_link"], inplace=True)
data_vk.rename(columns={
    "text": "Text",
    "vk_wall_id": "WallId",
    "vk_post_id": "PostId",
}, inplace=True)
data_vk["Source"] = "vk"

data_vk = data_vk.dropna()

data_vk.head()

,Text,WallId,PostId,Source
0,👌Новая поступления 🥳🥳🥳 \n Цена по ниже \n...,-192083416,9857,vk
1,💞💞💞💞💞💞💞💞💞💞💞💞\n💥💥Новое поступление💥💥\n\n🌹КОМПЛЕ...,-192083416,9858,vk
2,🌹ПОЛУЧИЛИ НОВЫЕ СОРОЧКИ\n🌹МАТЕРИАЛ 💯% ХЛОПОК\n...,-192083416,9860,vk
3,КУРТКИ ДЛИННЫЕ ЗИМНЕЕ \nРазмер 42-44-46-48-50\...,-192083416,9862,vk
4,ГРАНДИОЗНАЯ РАСПРОДАЖА \nТакова ещё не было на...,-192083416,9863,vk


In [ ]:
import re

def count_nums(string):
    return sum(re.match(r'.*[0-9]+', s) is not None for s in string.split())

data_vk["Nums"] = data_vk["Text"].apply(count_nums)

In [ ]:
data_vk.drop(labels=data_vk[data_vk["Nums"] == 0].index, inplace=True)
data_vk.drop(columns="Nums", inplace=True)

In [ ]:
data_tg = pd.read_csv("tg_final_filtered.csv")
data_tg.head()

,Text,Id,Length,Nums,Tokens,NumsPerWord,TokensPerWord,Wall
0,👕мужская рубашка primark\nразмер s-m\nсостояни...,"{""platform"": ""telegram"", ""wall_id"": 1332882905...",84,18,269,0.214286,3.202381,1332882905
1,"беговые кроссовки saucony men's axon 2, 44 раз...","{""platform"": ""telegram"", ""wall_id"": 1332882905...",16,5,50,0.312500,3.125000,1332882905
2,wifi repeater xiaomi \n15 gel. есть две штуки\...,"{""platform"": ""telegram"", ""wall_id"": 1332882905...",13,1,33,0.076923,2.538462,1332882905
3,и еще раз всем привет! продаю вафельницу wf 20...,"{""platform"": ""telegram"", ""wall_id"": 1332882905...",100,2,316,0.020000,3.160000,1332882905
4,"1. джинсы moms новые с бирками вайкики, маркир...","{""platform"": ""telegram"", ""wall_id"": 1332882905...",28,9,82,0.321429,2.928571,1332882905


In [ ]:
json.loads(data_tg.loc[0, "Id"])

{'platform': 'telegram', 'wall_id': 1332882905, 'post_id': 579955}

In [ ]:
data_tg.rename(columns={"Wall": "WallId"}, inplace=True)
data_tg["PostId"] = data_tg["Id"].apply(lambda json_str: json.loads(json_str)["post_id"])
data_tg.drop(columns=["Length", "Nums", "Tokens", "NumsPerWord", "TokensPerWord", "Id"], inplace=True)
data_tg["Source"] = "tg"

data_tg.head()

,Text,WallId,PostId,Source
0,👕мужская рубашка primark\nразмер s-m\nсостояни...,1332882905,579955,tg
1,"беговые кроссовки saucony men's axon 2, 44 раз...",1332882905,665316,tg
2,wifi repeater xiaomi \n15 gel. есть две штуки\...,1332882905,437718,tg
3,и еще раз всем привет! продаю вафельницу wf 20...,1332882905,477289,tg
4,"1. джинсы moms новые с бирками вайкики, маркир...",1332882905,335743,tg


In [ ]:
max_size = 50000
data = pd.concat([data_tg.iloc[:max_size, :], data_vk.iloc[:max_size, :]])
data = data.sample(frac=1)

data.head()

,Text,WallId,PostId,Source
7862,iРhonе 11 64гб в крacном цвeте с ЧЕКОM из магa...,-151775338,537819,vk
29325,"Kx-tca285\nБ/у недолго. В рабочем состоянии, н...",-48332904,704790,vk
1378,🌹ПОЛУЧИЛИ ДАЧНИ КОСТЮМ \n🌹КАЧЕСТВО СУПЕР \n🌹РА...,-192083416,12439,vk
16875,Продам 1000р. Размер 46. (В идеальном состояни...,-90442795,349691,vk
18594,мини-сумка charles&keith яичного цвета. новая....,1332882905,429778,tg


In [ ]:
data.to_csv("merged_final.csv", index=False)

In [ ]:
data.iloc[:100, :].to_csv("merged_random_batch.csv", index=False)